In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Preprocessing

# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN","NAME"])

# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9','T13','T12','T2','T25','T14','T29','T15','T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts()<1882].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Convert categorical data to numeric with `pd.get_dummies`
app_dummies = pd.get_dummies(application_df[['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']],dtype=int)

# Seperating numerical columns from categorical columns
int_cols = application_df[['STATUS','ASK_AMT','IS_SUCCESSFUL']]

# Creating a dataframe containing the numerical columns and the dummies
transformed_application_df = pd.concat([int_cols, app_dummies], axis=1)

In [3]:
# Split our preprocessed data into our features and target arrays
y = transformed_application_df.IS_SUCCESSFUL.values
X = transformed_application_df.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [4]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [5]:
# Optimization 1: Add more neurons to a hidden layer

# Define the model
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 160
hidden_nodes_layer2 = 60

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="relu")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 9s 8ms/step - loss: 0.5686 - accuracy: 0.7241
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5563 - accuracy: 0.7288
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5529 - accuracy: 0.7301
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5504 - accuracy: 0.7334
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5499 - accuracy: 0.7324
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5488 - accuracy: 0.7343
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5486 - accuracy: 0.7347
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5473 - accuracy: 0.7348
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5471 - accuracy: 0.7346
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accura

In [6]:
# Optimization 1: Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5611 - accuracy: 0.7255 - 535ms/epoch - 2ms/step
Loss: 0.5611070394515991, Accuracy: 0.7254810333251953


In [7]:
# Optimization 2: Add more hidden layers

# Define the model
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 9s 5ms/step - loss: 0.5702 - accuracy: 0.7171
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5546 - accuracy: 0.7248
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5518 - accuracy: 0.7310
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5504 - accuracy: 0.7325
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5499 - accuracy: 0.7330
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5486 - accuracy: 0.7322
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7332
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7334
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7329
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5463 - accura

In [8]:
# Optimization 2: Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5535 - accuracy: 0.7248 - 589ms/epoch - 2ms/step
Loss: 0.5534867644309998, Accuracy: 0.724781334400177


In [9]:
# Optimization 3: Increase the number of epochs

# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="relu")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 8s 8ms/step - loss: 0.5713 - accuracy: 0.7228
Epoch 2/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5554 - accuracy: 0.7301
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5524 - accuracy: 0.7309
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5514 - accuracy: 0.7320
Epoch 5/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5502 - accuracy: 0.7323
Epoch 6/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accuracy: 0.7324
Epoch 7/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5484 - accuracy: 0.7338
Epoch 8/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5476 - accuracy: 0.7347
Epoch 9/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accuracy: 0.7341
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accura

In [10]:
# Optimization 3: Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5665 - accuracy: 0.7254 - 492ms/epoch - 2ms/step
Loss: 0.5664948225021362, Accuracy: 0.7253644466400146


In [11]:
# Optimization 4: Decrease number of neurons

# Define the model
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 40
hidden_nodes_layer2 = 15

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5787 - accuracy: 0.7164
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5565 - accuracy: 0.7299
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5527 - accuracy: 0.7303
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7308
Epoch 5/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5502 - accuracy: 0.7318
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5490 - accuracy: 0.7333
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7346
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5480 - accuracy: 0.7343
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7338
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accura

In [12]:
# Optimization 4: Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5543 - accuracy: 0.7277 - 509ms/epoch - 2ms/step
Loss: 0.5542727112770081, Accuracy: 0.7276967763900757


In [18]:
# Optimization 5: Decrease the number of layers

# Define the model
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 3.2849 - accuracy: 0.6122
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 2.2904 - accuracy: 0.6538
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 2.0501 - accuracy: 0.6613
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 2.3372 - accuracy: 0.6515
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 2.7543 - accuracy: 0.6469
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 2.4056 - accuracy: 0.6560
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 2.1654 - accuracy: 0.6598
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 1.9846 - accuracy: 0.6775
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 2.0428 - accuracy: 0.6746
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 1.5240 - accura

In [19]:
# Optimization 5: Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7875 - accuracy: 0.7238 - 461ms/epoch - 2ms/step
Loss: 0.7875434756278992, Accuracy: 0.7238484025001526


In [15]:
# Optimization 6: Decrease the number of epochs

# Define the model
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=60)

Epoch 1/60
804/804 [==============================] - 5s 4ms/step - loss: 0.5703 - accuracy: 0.7232
Epoch 2/60
804/804 [==============================] - 3s 4ms/step - loss: 0.5563 - accuracy: 0.7285
Epoch 3/60
804/804 [==============================] - 3s 4ms/step - loss: 0.5526 - accuracy: 0.7316
Epoch 4/60
804/804 [==============================] - 2s 2ms/step - loss: 0.5519 - accuracy: 0.7326
Epoch 5/60
804/804 [==============================] - 2s 2ms/step - loss: 0.5502 - accuracy: 0.7327
Epoch 6/60
804/804 [==============================] - 2s 3ms/step - loss: 0.5493 - accuracy: 0.7331
Epoch 7/60
804/804 [==============================] - 3s 4ms/step - loss: 0.5485 - accuracy: 0.7324
Epoch 8/60
804/804 [==============================] - 2s 2ms/step - loss: 0.5473 - accuracy: 0.7350
Epoch 9/60
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accuracy: 0.7341
Epoch 10/60
804/804 [==============================] - 2s 2ms/step - loss: 0.5464 - accuracy: 0.7343

In [16]:
# Optimization 6: Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5529 - accuracy: 0.7258 - 443ms/epoch - 2ms/step
Loss: 0.552902340888977, Accuracy: 0.7258309125900269


In [17]:
0.7276967763900757 - 0.726880490779876

0.0008162856101996407